<a href="https://colab.research.google.com/github/ccsed/ProgettoLabAI/blob/main/Funzioni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collego google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importo librerie

In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 1.4 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import rasterio
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
import albumentations as A
from PIL import Image
import matplotlib.pyplot as plt
import random
import skimage.io as io
from rasterio.plot import show
import cv2

GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Funzione per dividere il dataset in training, validation e test set

In [ ]:
def split_dataset(image_dir, mask_dir, drive_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42):
    images = sorted(os.listdir(image_dir))
    masks = sorted(os.listdir(mask_dir))

    print(len(os.listdir(image_dir)))
    print(len(os.listdir(mask_dir)))

    # Controllo se il numero di immagini e maschere è lo stesso
    assert len(images) == len(masks), "Il numero di immagini e maschere non corrisponde"

    combined = list(zip(images, masks))
    random.seed(seed)
    random.shuffle(combined)

    train_split = int(len(combined) * train_ratio)
    val_split = int(len(combined) * (train_ratio + val_ratio))
    train_data = combined[:train_split]
    val_data = combined[train_split:val_split]
    test_data = combined[val_split:]

    # Scrivere i percorsi dei file nelle liste train, val e test
    train_files = [(os.path.join(image_dir, img), os.path.join(mask_dir, lbl)) for img, lbl in train_data]
    val_files = [(os.path.join(image_dir, img), os.path.join(mask_dir, lbl)) for img, lbl in val_data]
    test_files = [(os.path.join(image_dir, img), os.path.join(mask_dir, lbl)) for img, lbl in test_data]

    # Scrivere i percorsi dei file nei file .txt
    with open(os.path.join(drive_dir, 'train.txt'), 'w') as f:
        for img_path, lbl_path in train_files:
            f.write(f"{img_path} {lbl_path}\n")

    with open(os.path.join(drive_dir, 'val.txt'), 'w') as f:
        for img_path, lbl_path in val_files:
            f.write(f"{img_path} {lbl_path}\n")

    with open(os.path.join(drive_dir, 'test.txt'), 'w') as f:
        for img_path, lbl_path in test_files:
            f.write(f"{img_path} {lbl_path}\n")


In [2]:
import os
import random
import shutil
from tqdm import tqdm

def split_dataset(image_dir, mask_dir, drive_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42):
    images = sorted(os.listdir(image_dir))
    masks = sorted(os.listdir(mask_dir))

    print(f"Numero di immagini: {len(images)}")
    print(f"Numero di maschere: {len(masks)}")


    assert len(images) == len(masks), "Il numero di immagini e maschere non corrisponde"

    combined = list(zip(images, masks))
    random.seed(seed)
    random.shuffle(combined)

    train_split = int(len(combined) * train_ratio)
    val_split = int(len(combined) * (train_ratio + val_ratio))
    train_data = combined[:train_split]
    val_data = combined[train_split:val_split]
    test_data = combined[val_split:]


    for folder in ['train_images', 'train_masks', 'val_images', 'val_masks', 'test_images', 'test_masks']:
        os.makedirs(os.path.join(drive_dir, folder), exist_ok=True)

    def move_files(file_list, image_dest, mask_dest, description):
        for img, lbl in tqdm(file_list, desc=description):
            shutil.move(os.path.join(image_dir, img), os.path.join(image_dest, img))
            shutil.move(os.path.join(mask_dir, lbl), os.path.join(mask_dest, lbl))


    move_files(train_data, os.path.join(drive_dir, 'train_images'), os.path.join(drive_dir, 'train_masks'), 'Spostamento Train')
    move_files(val_data, os.path.join(drive_dir, 'val_images'), os.path.join(drive_dir, 'val_masks'), 'Spostamento Val')
    move_files(test_data, os.path.join(drive_dir, 'test_images'), os.path.join(drive_dir, 'test_masks'), 'Spostamento Test')




Definisco alcune directory

In [3]:
drive_dir = '/content/drive/MyDrive'
image_dir = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/SAR-Intensity'
train_path = '/content/drive/MyDrive/train.txt'
val_path = '/content/drive/MyDrive/val.txt'
test_path = '/content/drive/MyDrive/test.txt'
label_dir = drive_dir + '/Labels'

Faccio split del dataset

In [ ]:
split_dataset(image_dir, label_dir, drive_dir)

Numero di immagini: 3401
Numero di maschere: 3401


Spostamento Test: 100%|██████████| 341/341 [00:01<00:00, 190.34it/s]


In [4]:
import os
import shutil
from tqdm import tqdm

def restore_dataset(original_image_dir, original_mask_dir, split_dir):
    # Directory di origine dei file suddivisi
    split_folders = {
        'train_images': original_image_dir,
        'train_masks': original_mask_dir,
        'val_images': original_image_dir,
        'val_masks': original_mask_dir,
        'test_images': original_image_dir,
        'test_masks': original_mask_dir
    }

    # Funzione per spostare i file indietro con barra di avanzamento
    def move_files_back(src_dir, dest_dir, description):
        files = os.listdir(src_dir)
        for file in tqdm(files, desc=description):
            shutil.move(os.path.join(src_dir, file), os.path.join(dest_dir, file))

    # Spostare i file indietro
    for folder, dest in split_folders.items():
        src_dir = os.path.join(split_dir, folder)
        move_files_back(src_dir, dest, f"Spostamento {folder}")


restore_dataset(image_dir, label_dir, drive_dir)


Spostamento train_images: 0it [00:00, ?it/s]
Spostamento train_masks: 0it [00:00, ?it/s]
Spostamento val_images: 0it [00:00, ?it/s]
Spostamento val_masks: 0it [00:00, ?it/s]
Spostamento test_images: 0it [00:00, ?it/s]
Spostamento test_masks: 0it [00:00, ?it/s]


Path delle immagini

In [ ]:
with open(train_path, 'r') as f:
    train_data = [line.split() for line in f.read().splitlines()]

train_image_paths = [line[0] for line in train_data]

with open(val_path, 'r') as f:
    val_data = [line.split() for line in f.read().splitlines()]

val_image_paths = [line[0] for line in val_data]

with open(test_path, 'r') as f:
    test_data = [line.split() for line in f.read().splitlines()]

test_image_paths = [line[0] for line in test_data]

Funzione per calcolare media e deviazione standard sul training set

In [ ]:
def compute_mean_std(image_paths):
    mean = np.zeros(4)
    std = np.zeros(4)
    nb_samples = 0

    # Abilita l'uso della GPU per OpenCV
    cv2.setUseOptimized(True)
    cv2.setNumThreads(8)
    cv2.ocl.setUseOpenCL(True)

    for image_path in image_paths:
        try:
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

            for i in range(4):
                mean[i] += np.mean(image[:,:,i])
                std[i] += np.std(image[:,:,i])

            nb_samples += 1

            if nb_samples % 10 == 0:
                print(f"Processed {nb_samples} images")

        except Exception as e:
            print(f"Error loading image: {image_path}")
            print(e)
            continue

    mean /= nb_samples
    std /= nb_samples

    return mean, std

Calcolo media e std su training set

In [ ]:
mean, std = compute_mean_std(train_image_paths)
print("Mean: ", mean)
print("Std: ", std)

Processed 10 images
Processed 20 images
Processed 30 images
Processed 40 images
Processed 50 images
Processed 60 images
Processed 70 images
Processed 80 images
Processed 90 images
Processed 100 images
Processed 110 images
Processed 120 images
Processed 130 images
Processed 140 images
Processed 150 images
Processed 160 images
Processed 170 images
Processed 180 images
Processed 190 images
Processed 200 images
Processed 210 images
Processed 220 images
Processed 230 images
Processed 240 images
Processed 250 images
Processed 260 images
Processed 270 images
Processed 280 images
Processed 290 images
Processed 300 images
Processed 310 images
Processed 320 images
Processed 330 images
Processed 340 images
Processed 350 images
Processed 360 images
Processed 370 images
Processed 380 images
Processed 390 images
Processed 400 images
Processed 410 images
Processed 420 images
Processed 430 images
Processed 440 images
Processed 450 images
Processed 460 images
Processed 470 images
Processed 480 images
P

Funzioni per convertire i file geojson in tiff

In [ ]:
import os
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
import numpy as np
import matplotlib.pyplot as plt

def create_mask_from_labels(geojson_path, reference_image_path, output_tiff_path):
    try:
        with rasterio.open(reference_image_path) as src:
            transform = src.transform
            out_shape = (src.height, src.width)
    except Exception as e:
        raise ValueError(f"Error reading reference TIFF file {reference_image_path}: {e}")

    try:
        gdf = gpd.read_file(geojson_path)
    except Exception as e:
        raise ValueError(f"Error reading GeoJSON file {geojson_path}: {e}")

    if gdf.empty or gdf.geometry.isnull().all():
        mask = np.zeros(out_shape, dtype='uint8')
    else:
        mask = rasterize(
            [(geom, 255) for geom in gdf.geometry if geom.is_valid],
            out_shape=out_shape,
            transform=transform,
            fill=0,
            dtype='uint8'
        )

    with rasterio.open(
        output_tiff_path, 'w',
        driver='GTiff',
        height=mask.shape[0],
        width=mask.shape[1],
        count=1,
        dtype=mask.dtype,
        crs='+proj=latlong',
        transform=transform,
    ) as dst:
        dst.write(mask, 1)


def process_dataset(geojson_directory, reference_image_directory, output_tiff_directory):

    if not os.path.exists(output_tiff_directory):
        os.makedirs(output_tiff_directory)
    geojson_files = [f for f in os.listdir(geojson_directory) if f.endswith('.geojson')]
    count = 0
    for geojson_file in geojson_files:
        count += 1
        if count%20 == 0:
           print(count)
        geojson_path = os.path.join(geojson_directory, geojson_file)
        reference_image_path = os.path.join(reference_image_directory, geojson_file.replace('.geojson', '.tif').replace('Buildings', 'PS-RGB'))
        output_tiff_path = os.path.join(output_tiff_directory, geojson_file.replace('.geojson', '_mask.tiff'))

        if os.path.exists(output_tiff_path):
            print(f"{output_tiff_path} already exists, skipping...")
            continue

        if not os.path.exists(reference_image_path):
            print(f"Reference image not found for {geojson_file}, skipping...")
            continue

        try:
            create_mask_from_labels(geojson_path, reference_image_path, output_tiff_path)
            print(f"Converted {geojson_file} to {output_tiff_path}")
        except Exception as e:
            print(f"Failed to convert {geojson_file}: {e}")


In [ ]:
geojson_dir = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/geojson_buildings'
rgb_dir = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/PS-RGB'
output_dir = '/content/drive/MyDrive/Labels'

process_dataset(geojson_dir, rgb_dir, output_dir)